<a href="https://colab.research.google.com/github/kmk4444/Autonomic_agents/blob/main/Part2_advanced_react.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will develop advanced react application.
There are three language models.
There are two search models.
There are two creating image models.

**Requirements.txt**

In [34]:
!touch requirements.txt
!echo langchain >> requirements.txt
!echo langchain-openai >> requirements.txt
!echo langchain-google-genai >> requirements.txt
!echo langchainhub >> requirements.txt
!echo python-dotenv >> requirements.txt
!echo streamlit >> requirements.txt
!echo duckduckgo-search >> requirements.txt
!echo anthropic >> requirements.txt
!echo beautifulsoup4 >> requirements.txt
!echo autogenstudio >> requirements.txt
!echo crewai >> requirements.txt

**terminal / bash komutu**

In [35]:
pip install -r requirements.txt

In [46]:
%%writefile customtools.py

from langchain.agents import Tool # It is the most important package
from openai import OpenAI #for dalle3
from bs4 import BeautifulSoup # web scraping
from io import BytesIO # image processing
import base64 # image processing
import requests # request for remote server
from datetime import datetime # picture name
import os
from dotenv import load_dotenv

#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
#my_key_stabilityai = os.getenv("stabilityai_apikey")

my_key_openai="---"
my_key_stabilityai="----"

client = OpenAI(
    api_key=my_key_openai
)

#SDXL
#DALLE3
#Bsoup
#Method->Tool

def generate_image_with_dalle(prompt):

  AI_Response = client.images.generate(
      model="dall-e-3",
      size="1024x1024",
      quality="hd",
      n=1, # the number of picture
      response_format = "url", #answer will be url
      prompt=prompt
  )

  image_url = AI_Response.data[0].url # we arranged answer of chatbot

  response = requests.get(image_url) # dowloand picture
  image_bytes=BytesIO(response.content) # we assign content of picture to image_bytes using BytesIO


  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") # open filepath
  filepath = f"/content/sample_data/generated_image_{timestamp}.png" # write file inside of filepath

  #if not os.path.exists("./img"):
  #    os.makedirs("./img")

  with open(filepath,"wb") as file:
    file.write(image_bytes.getbuffer())

  return f'<a href="{filepath}">Resminiz Burada</a>' # it creates a link as html format

#We call tools.
#The get_tool function takes a selected_image_generator argument. This argument is a string representing the image generator chosen by the user.
def get_tool(selected_image_generator):

    if selected_image_generator == "DALL-E 3":
        return Tool (
            name="Generate Image", #Generate Image" (name of the returned tool)
            func= generate_image_with_dalle, # (image generation function)
            description="""Useful for when you need to generate an image based on given textual instructions or prompts.
            It returns the filepath where the image saved. This filepath must be given back to the user.
            The filepath is given between HTML tags for turning the address of the image into a link.
            The user must be provided with this HTML style statement that includes the filepath for the saved image.
            """ # A description of what the tool does
        )

    elif selected_image_generator == "Stable Diffusion XL":
        return Tool (
            name="Generate Image", #Generate Image" (name of the returned tool)
            func= generate_with_SD, # (image generation function)
            description="""Useful for when you need to generate an image based on given textual instructions or prompts.
            It returns the filepath where the image saved. This filepath must be given back to the user.
            The filepath is given between HTML tags for turning the address of the image into a link.
            The user must be provided with this HTML style statement that includes the filepath for the saved image.
            """ # A description of what the tool does
        )

#BeautifulSoup provides to create webpage content
def analyze_webpage(target_url): # Defines a function to analyze a webpage given its URL.

  response = requests.get(target_url) # Fetches the HTML content from the target URL.
  html_content = response.text # Stores the fetched HTML content as text.

  soup = BeautifulSoup(html_content, "html.parser") # Stores the fetched HTML content as text.
  stripped_content = soup.get_text() # Extracts all the text content from the parsed HTML structure.

  # It helps to short long content because we don't know content of webpage. Maybe it is too long;that's why, we will use first 4000 characters.
  if len(stripped_content) >4000:
    stripped_content = stripped_content[:4000]

  return stripped_content

# Right now, we create tool for analyze_webpage
def get_web_tool():
  return Tool(
      name="Get Webpage",
      func=analyze_webpage,
      description="Useful for when you need to get the http from a specific webpage"
  )


Overwriting customtools.py


In [51]:
%%writefile app.py

#load_tools is duckduckgo search method.
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_models import ChatAnthropic
from langchain import hub
from langchain_community.callbacks import StreamlitCallbackHandler # it shows agent processes on screen (reasoning and acting)
from langchain_community.tools.tavily_search import TavilySearchResults
import streamlit as st
import os
#import customtools
from dotenv import load_dotenv


#load_dotenv()

#my_key_openai = os.getenv("openai_apikey")
#my_key_google = os.getenv("google_apikey")
#my_key_anthropic = os.getenv("anthropic_apikey")
#os.environ["TAVILY_API_KEY"] = os.getenv("tavily_apikey")

my_key_openai="----"
#my_key_google="---"
my_key_anthropic="---"
os.environ["TAVILY_API_KEY"]="----"

#llm_gemini = ChatGoogleGenerativeAI(google_api_key=my_key_google, model="gemini-1.0-pro") ERROR!
llm_gpt = ChatOpenAI(api_key=my_key_openai, model="gpt-4-0125-preview", temperature=0, streaming=True)
llm_claude = ChatAnthropic(anthropic_api_key=my_key_anthropic, model_name="claude-2.1")

#System prompt, we call system prompt from langchain hub.
agent_prompt = hub.pull("hwchase17/react")

def configure_agent(selected_llm, selected_search_engine, selected_image_generator=""):
    #
    if selected_llm == "GPT-4":
        llm = llm_gpt
    elif selected_llm == "Gemini Pro":
        llm = llm_gemini
    elif selected_llm == "Claude 2.1":
        llm = llm_claude

    image_generator_tool = customtools.get_tool(selected_image_generator=selected_image_generator)
    web_scraping_tool = customtools.get_web_tool()

    if selected_search_engine =="DuckDuckGo":
      tools = load_tools(["ddg-search"]) # we call load_tools method to run duckduckgo
      tools.extend([image_generator_tool, web_scraping_tool])
    elif selected_search_engine == "Tavily":
      tools = [TavilySearchResults(max_results=1), image_generator_tool, web_scraping_tool] # we reach one best solution. you can increase max_results but sometimes it can be error because of langchain mistake.


    agent = create_react_agent(llm=llm, tools=tools, prompt=agent_prompt) # we described agent
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True) #we need executor to run agent
    #verbose=True provides colorful output.

    return agent_executor


#Interface

st.set_page_config(page_title="ReAct Ajan ile Sohbet Etkileşimi")
#st.image(image="./img/ai_agent_banner.png")
st.title("ReAct Ajan ile Sohbet Etkileşimi")
#st.divider()

st.sidebar.header("Ajan Konfigürasyonu")
st.sidebar.divider()
selected_llm = st.sidebar.radio(label="Dil Modeli Seçiniz", options=["GPT-4", "Gemini Pro", "Claude 2.1"])
st.sidebar.divider()
selected_search_engine = st.sidebar.radio(label="Arama Motoru Seçiniz", options=["DuckDuckGo", "Tavily"], index=1)
st.sidebar.divider()
selected_image_generator = st.sidebar.radio(label="Resim Üretim Modelini Seçiniz", options=["Stable Diffusion XL","DALL-E 3"])
st.sidebar.divider()
selected_web_scraper = st.sidebar.radio(label="Web Kazıma Aracı Seçiniz", options=["BeautifulSoup"])
st.sidebar.divider()
turkish_sensitivity = st.sidebar.checkbox(label="Türkçe Yanıta Zorla", value=True)
st.sidebar.divider()
reset_chat_btn = st.sidebar.button(label="Sohbeti Geçmişini Sıfırla")


# We want to save old messages on the screen.
# When we run program first time, program will create empty message list.
if "messages" not in st.session_state: # if messages is not created, we'll create Session State message list.
  st.session_state.messages = []
  #st.session_state.messages.append({"role":"system", "content":"Sen yardımsever bir asistansın"}) # We append system default prompt. (This is default first message)


#we will show our message on the chat window.
for message in st.session_state.messages:
  with st.chat_message(message["role"]):# it provides to show icon for role,asisstant and system.
    st.markdown(message["content"])# it provides to show icon for role,asisstant and system.

#if user click prompt button
if prompt := st.chat_input(placeholder="Mesajınızı yazınız"):
  st.chat_message("user").write(prompt) # thanks to chat_message, user prompt will be shown on the screen.

  # if turkish is selected, we add a little sentence to user prompt.
  if turkish_sensitivity:
    st.session_state.messages.append({"role":"user", "content": prompt + "Bu soruyu Türkçe yanıtla"})
  else:
      st.session_state.messages.append({"role":"user", "content": prompt})

  #agent create an answer.
  with st.chat_message("assistant"):
    st.info("🧠 Düşünce Zinciri İşletiliyor...")

    # it shows all process of agent;that is to say, it shows terminal screen on your web page.
    st_callback = StreamlitCallbackHandler(st.container())
    # we start agent accordingly our prompt, engine and image.
    executor = configure_agent(selected_llm=selected_llm, selected_search_engine=selected_search_engine, selected_image_generator=selected_image_generator)

    # LLM create an answer using invoke method.
   # callbacks provide better user friend demonstration of process of agent.
    AI_Response = executor.invoke(
        {"input": st.session_state.messages}, {"callbacks": [st_callback]}, # Eski mesajlarıda dikkate alarak cevap verecek. we used st.session_state.messages instead of prompt because we want to answer from gpt using all historical messages and new message.
        handle_parsing_errors=True #handle_parsing_errors=True means when agents work together, there can be error. Thanks to that, program ignores these errors.
        )
    #unsafe_allow_html=True, we can click link if chatbot show a webpage or we can see graph or picture
    #AI_Response["output"] we select output keyword of AI_response array.
    st.markdown(AI_Response["output"], unsafe_allow_html=True)

    st.session_state.messages.append({"role":"assistant", "content": AI_Response["output"]})

#we deleted history chat
if reset_chat_btn:
    st.session_state.messages = []
    st.toast("Sohbet geçmişi sıfırlandı!")


Overwriting app.py


In [58]:
! pip install streamlit==1.25.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.6 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.93s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details
npx: installed 22 in 4.276s
your url is: https://tender-animals-fetch.loca.lt
